# Phase 1: Data Loading

**Objective:** Connect to Supabase, fetch the `ventas_raw` table, and save the raw data locally.

**Outputs:**
- Raw Data: `data/01_raw/ventas_raw.csv`
- Reports: `outputs/reports/p1_load_stats.json`, `outputs/reports/p1_loading_report.md`

In [1]:
import sys
import os
import pandas as pd
import json
from dotenv import load_dotenv

# Add project root to path
sys.path.append(os.path.abspath('..'))

from src.connectors.loader import SupabaseLoader

In [2]:
# Initialize Loader
loader = SupabaseLoader()
print("Supabase Loader Initialized")

Supabase Loader Initialized


In [3]:
# Fetch Data
TABLE_NAME = "ventas_raw"
df = loader.fetch_data(TABLE_NAME)
print(f"Data loaded: {df.shape}")
df.head()

Fetching data from table: ventas_raw...
Fetched 1000 rows...
Fetched 2000 rows...
Fetched 3000 rows...
Fetched 4000 rows...
Fetched 5000 rows...
Fetched 6000 rows...
Fetched 7000 rows...
Fetched 8000 rows...
Fetched 9000 rows...
Fetched 10000 rows...
Fetched 11000 rows...
Fetched 12000 rows...
Fetched 13000 rows...
Fetched 14000 rows...
Fetched 15000 rows...
Fetched 16000 rows...
Fetched 17000 rows...
Fetched 18000 rows...
Fetched 19000 rows...
Fetched 20000 rows...
Fetched 21000 rows...
Fetched 22000 rows...
Fetched 23000 rows...
Fetched 24000 rows...
Fetched 25000 rows...
Fetched 26000 rows...
Fetched 27000 rows...
Fetched 28000 rows...
Fetched 29000 rows...
Fetched 30000 rows...
Fetched 30099 rows...
Total rows fetched: 30099
Data loaded: (30099, 11)


,id,fecha,region,punto_venta,metodo_pago,tipo,unidades_vendidas,precio_venta,ingresos_totales,costo_unitario,created_at
0,1,2018-01-01,Manizales,Delivery,Efectivo,Carne,422,1800,759600,1300,2026-01-28T19:33:19.512953+00:00
1,2,2018-01-01,Manizales,Delivery,Efectivo,Pollo,293,1650,483450,1150,2026-01-28T19:33:19.682927+00:00
2,3,2018-01-01,Manizales,Delivery,Efectivo,Queso,156,1450,226200,1000,2026-01-28T19:33:19.845935+00:00
3,4,2018-01-01,Manizales,Delivery,Efectivo,Mixta,118,2000,236000,1400,2026-01-28T19:33:19.99027+00:00
4,5,2018-01-01,Manizales,Delivery,Efectivo,Chicharrón,95,1950,185250,1350,2026-01-28T19:33:20.147215+00:00


In [4]:
# Ensure data directory exists
os.makedirs("../data/01_raw", exist_ok=True)

# Save Raw Data
raw_path = "../data/01_raw/ventas_raw.csv"
df.to_csv(raw_path, index=False)
print(f"Raw data saved to {raw_path}")

Raw data saved to ../data/01_raw/ventas_raw.csv


### Generate Reporting Artifacts

In [5]:
# 1. JSON Stats
stats = {
    "rows": int(df.shape[0]),
    "columns": int(df.shape[1]),
    "columns_list": df.columns.tolist(),
    "date_min": str(df['fecha'].min()) if 'fecha' in df.columns else None,
    "date_max": str(df['fecha'].max()) if 'fecha' in df.columns else None,
    "nulls": df.isnull().sum().to_dict()
}

reports_dir = "../outputs/reports"
os.makedirs(reports_dir, exist_ok=True)

with open(f"{reports_dir}/p1_load_stats.json", "w") as f:
    json.dump(stats, f, indent=4)
    
print("Stats JSON saved.")

Stats JSON saved.


In [6]:
# 2. Markdown Report
report_content = f"""# Fase 1: Reporte de Carga de Datos (Data Ingestion)

## 1. Principales Hallazgos
- **Volumen de Datos**: Se cargaron exitosamente **{stats['rows']} registros** provenientes de la tabla `{TABLE_NAME}` de Supabase.
- **Estructura**: El dataset cuenta con **{stats['columns']} columnas**, incluyendo dimensiones geográficas, temporales y financieras.
- **Rango Temporal**: Los datos cubren desde el **{stats['date_min']}** hasta el **{stats['date_max']}**.
- **Integridad Inicial**: No se detectaron valores nulos técnicos en la carga inicial.

## 2. Principales Inconvenientes
- **Valores Centinela**: Se requiere limpieza de valores 0 o negativos en unidades/precios en la Fase 2.
- **Paginación**: La carga masiva requiere gestión de offsets para evitar timeouts de la API.

## 3. Sugerencias
- **Optimización de Formato**: Considerar Parquet para mejorar el rendimiento de lectura.
- **Monitoreo de Drift**: Implementar alertas si la carga diaria cae significativamente.

## 4. Recomendaciones
- **Fase de Transformación**: Iniciar la limpieza siguiendo las reglas de negocio de valores centinela numéricos y categóricos.
- **Resguardo de Crudos**: No modificar `ventas_raw.csv` directamente.

## 5. Conclusiones
La ingestión de datos ha sido exitosa. El dataset es íntegro y el volumen es suficiente para iniciar el análisis y modelado.
"""

with open(f"{reports_dir}/p1_loading_report.md", "w", encoding='utf-8') as f:
    f.write(report_content)

print("Markdown Report saved.")

Markdown Report saved.
